In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from flowdec.nb import utils as nbutils 
from flowdec import data as fd_data
import pims
# import pims
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
from flowdec import psf as fd_psf
import dask
import dask.array as da
import tifffile as tf
from nd2reader import ND2Reader
# from pims import ND2_Reader as ND2Reader
# import aicsimageio.vendor.omexml as ome
import glob

In [2]:
# channels = ['miRFP670', 'mRuby3','FITC']
import time, sys
from IPython.display import clear_output

sys.path.insert(0,'/home/jmamede/scripts/notebooks/libraries/')
from deco_libraries import update_progress, pimsmeta2OMEXML, Nd2meta2OMEXML, observer, observer, init_RL_algo, deconv

In [3]:
#THINGS TO CHANGE!!!!! Always keep /*.nd2 at the end

dirname = "/home/jmamede/Data/CaRuby3/VOG/*"
# dirname = "D:/JM/20201222caru3/Carubyigfp_F2_Nh4cl_vitc_rutin_BVD002.nd2"

filelist = glob.glob(dirname)
filelist.sort()
# filelist = filelist[3:]
print(filelist)

frames = pims.bioformats.BioformatsReader(filelist[0],java_memory='1024m')
print(frames.sizes)

metadata = frames.metadata
print(metadata.ChannelEmissionWavelength(0,0))

ydivide = 1
xdivide = 1

if xdivide >1 and ydivide >1:
    depthdivide = (0,64,64)
elif xdivide > 1 and ydivide == 1:
        depthdivide = (0,0,64)
else:
        depthdivide = (0,0,0)
# print(frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'ppNextLevelEx'][b''][b'ppNextLevelEx'][b''][b'uLoopPars'][b'dZStep'])


['/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon.nd2', '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v1.ome.tiff', '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v1_PRJ.ome.tiff', '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v2.ome.tiff', '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v2_PRJ.ome.tiff', '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v3.ome.tiff', '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v3_PRJ.ome.tiff', '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v4.ome.tiff', '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v4_PRJ.ome.tiff', '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v5.ome.tiff', '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v5_PRJ.ome.tiff', '/home/jmamede/Data/CaRuby3/VOG/Caruby_PSPAX.nd2', '/home/jmamede/Data/CaRuby3/VOG/iGFP_Caru3_3_3.nd2']
{'x': 1024, 'y': 1024, 'c': 3, 'z': 7}
523.0


/home/jmamede/anaconda3/lib/python3.7/site-packages/pims/bioformats.py:409: UserWarning: Due to an issue with JPype 0.6.0, reading is slower. Please consider upgrading JPype to 0.6.1 or later.
  warn('Due to an issue with JPype 0.6.0, reading is slower. '


In [4]:
frames

<FramesSequenceND>
Axes: 4
Axis 'x' size: 1024
Axis 'y' size: 1024
Axis 'c' size: 3
Axis 'z' size: 7
Pixel Datatype: <u2

In [5]:
PSFdir="/home/jmamede/Rush/PSF/"
psf647 = tf.imread(PSFdir+"/AF647_0.3_EMCCD.tif")
psfruby =  tf.imread(PSFdir+'/mRuby3_0.3_EMCCD.tif')
psfgfp =  tf.imread(PSFdir+'/green_0.3_EMCCD.tif')
psfdapi =  tf.imread(PSFdir+'/DAPI_0.3_EMCCD.tif')
# psf647 = tf.imread(PSFdir+"/AF647_0.5_EMCCD.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3_0.5_EMCCD.tif')
# psfgfp =  tf.imread(PSFdir+'/green_0.5_EMCCD.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI_0.5_EMCCD.tif')
# psf647 = tf.imread(PSFdir+"/AF647.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3.tif')
# psfgfp =  tf.imread(PSFdir+'/green.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI.tif')
# psfruby = psf647
# psfgfp = psf647
# psfdapi = psf647
remove = 8
remove = 14
psf647 = np.flip(psf647,
#                  axis=0
                )[remove:-remove]
# [8:-8]


psfgfp = np.flip(psfgfp,
#                  axis=0
                )[remove:-remove]
psfdapi = np.flip(psfdapi,
#                   axis=0
                 )[remove:-remove]
psfruby = np.flip(psfruby,
#                   axis=0
                 )[remove:-remove]

print(psf647.shape)
print(psfruby.shape)
# psf647

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


(7, 128, 128)
(7, 128, 128)


In [6]:
algo =init_RL_algo(psfgfp.ndim
#                    ,pad_mode='2357',pad_min=(1,1,1)
                   ,pad_mode='log2',pad_min=(1,1,1)
                  )
filelist

['/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon.nd2',
 '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v1.ome.tiff',
 '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v1_PRJ.ome.tiff',
 '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v2.ome.tiff',
 '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v2_PRJ.ome.tiff',
 '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v3.ome.tiff',
 '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v3_PRJ.ome.tiff',
 '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v4.ome.tiff',
 '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v4_PRJ.ome.tiff',
 '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v5.ome.tiff',
 '/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon_v5_PRJ.ome.tiff',
 '/home/jmamede/Data/CaRuby3/VOG/Caruby_PSPAX.nd2',
 '/home/jmamede/Data/CaRuby3/VOG/iGFP_Caru3_3_3.nd2']

In [7]:
for fname in filelist:
    frames =  pims.bioformats.BioformatsReader(fname,java_memory='1024m')
#     frames.iter_axes = 't'  # 't' is the default already
    frames.bundle_axes = 'zyx'  # when 'z' is available, this will be default

    channels = [frames.metadata.ChannelEmissionWavelength(0,ch) for ch in range(frames.metadata.ChannelCount(0))]
    metadata = frames.metadata
    xml = pimsmeta2OMEXML(frames)
    prjxml = pimsmeta2OMEXML(frames, project=True)

#     psf =[]

    chunk_size=(frames.sizes['z'],frames.sizes['y']//ydivide,frames.sizes['x']//xdivide)


    # frames.iter_axes = 'v'
    # for frame in frames:
    #update SERIES HERE
    nseries = metadata.ImageCount()
    for visit in range(nseries):
#     for visit in range(21,31):

        frames.series = visit
        stackfile = fname[:-4]+"_v"+str(visit+1)+'.ome.tiff'
        prjfile = fname[:-4]+"_v"+str(visit+1)+'_PRJ.ome.tiff'
        
        with tf.TiffWriter(stackfile, bigtiff=True ,
                           imagej=False) as tif,  tf.TiffWriter(
                        prjfile, bigtiff=True, imagej=False) as tifprj:
            first = True
#             for time in range(frames.sizes['t']):
            for time in range(1):

                res = np.zeros(
                    (frames.sizes['z'],frames.sizes['c']
                     ,frames.sizes['y'],frames.sizes['x'])
                    , dtype=np.float32)
                for i in range(frames.sizes['c']):
                    frames.default_coords['c'] = i
#                     print('Channel:',metadata['channels'][i])
                    ch = str(metadata.ChannelEmissionWavelength(0,i))
                    # ['a647', 'DAPI', 'GFP']
                    # ['FITC', 'mRuby3', 'miRFP670']
                    # ['FRET-gYFP-dsRED', 'mRuby', 'GFP', 'DAPI']
                    # ['a647', 'DAPI', 'GFP']
                    if ch == 'DAPI' or ch  == '470 nm':
                        psf = psfdapi
                    elif ch == 'miRFP670' or ch == 'a647' or ch == 'mirfp670' or ch == 'farRED-EM' or ch == '670.0':
                        psf = psf647
                    elif ch == 'GFP' or ch == 'FITC'  or ch == 'GREEN-EM' or  ch == '535.0':
                        psf = psfgfp
                    elif ch == 'mRuby3' or ch == 'mRuby' or ch ==  '555 nm' or ch == 'RED-EM' or ch == '590.0':
                        psf = psfruby
                    elif ch == 'FRET-gYFP-dsRED':
                        psf = psfruby
                    elif ch == '548.0':
                        psf = psfyfp
                    elif ch == '625.0' or ch == '626.0':
                        psf = psfch
                    else: psf = psfruby

                    arr = da.from_array(frames[time]
    #                                     [:,0:2044,0:2044]
    #                                     [:,:,:]              
                                        , chunks=chunk_size)

#                     print(arr.shape)
                    res[:,i,:,:] = da.map_overlap(deconv,arr, depth  = depthdivide ,boundary='reflect',
                                        dtype='float32', algo=algo,
                                        psf=psf, iters=20, use_ram=1
                                                 ).compute(num_workers=1)
#                     res[:,i,:,:] = deconv(frames[time],
# #                                           depth  = depthdivide, boundary='reflect', dtype='float32',
#                                           algo=algo, psf=psf, iters=20
#                              )
#     .compute(num_workers=1)
                    # res[i,:,:,:] = arr.map_blocks(
                    # deconv,dtype='float32').compute(num_workers=1)

                    # If we need to swap Channel and Z in the future
                    #             img5d = np.swapaxes(img5d,0,1)

#                     update_progress( (visit+1) * (time+1) / (nseries * frames.sizes['t']) )
                    update_progress( (visit+1) * (time+1) / (nseries ) )
                    print("Visit Point: {}/{} \t Time:{}/{} \t Channel:{}/{}".format(
                            visit+1, nseries
                            ,time+1, 1
#                         ,time+1, frames.sizes['t']
                            , i+1, frames.sizes['c']))

                if first:
#                     tif.save(res.astype(np.uint16)
# #                             , compress='LZMA'
#                         , description = xml.to_xml()
#                         , photometric='minisblack'
#                         #, datetime= True
#                         , metadata= None
#                         , contiguous=False
#                         )
                    tifprj.save(res.astype(np.uint16).max(axis=0),
#                             , compress='LZMA'
                        description = prjxml.to_xml(),
                        photometric='minisblack',
                        #, datetime= True
                        metadata= None,
                        contiguous=False
                        )
                    first = False
                else:
#                     tif.save(res.astype(np.uint16)
# #                             , compress='LZMA'
# #                         , description = xml.to_xml()
#                         , photometric='minisblack'
#                         , metadata= None
#                         , contiguous=False
#                         )
                    tifprj.save(res.astype(np.uint16).max(axis=0),
#                             , compress='LZMA'
#                         , description = xml.to_xml()
                        photometric='minisblack',
                        #, datetime= True
                        metadata= None,
                        contiguous=False,
                        )

        tif.close()
        tifprj.close()

    update_progress(1)

Progress: [####################] 100.0%


loci.formats.FormatException: loci.formats.FormatException: No IFDs found

In [17]:
# print('done')
from numba import cuda
cuda.select_device(0)
cuda.close()
#the memory was released here!
cuda.select_device(0)

<weakproxy at 0x7f2e7c1a8230 to Device at 0x7f2e7c387fd0>